# Grafo de proximidad geográfica entre países

Este notebook carga el dataset `data/covid_19_country_daily_with_coords.csv` y construye un grafo totalmente conectado
entre países. Cada arista lleva un peso normalizado en el rango [0, 1] que refleja la cercanía geográfica
(1 significa países coincidentes/muy cercanos y valores cercanos a 0 corresponden a países muy distantes).


In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from itertools import combinations
from math import radians, sin, cos, sqrt, atan2

print("Librerías cargadas correctamente")


Librerías cargadas correctamente


In [2]:
DATA_PATH = "data/covid_19_country_daily_with_coords.csv"

df = pd.read_csv(DATA_PATH)
print(f"Registros cargados: {len(df):,}")
df.head()


Registros cargados: 87,279


,Country/Region,ObservationDate,Confirmed,Deaths,Recovered,Latitud_promedio,Longitud_promedio
0,Azerbaijan,02/28/2020,1.0,0.0,0.0,40.14310,47.576900
1,Afghanistan,01/01/2021,51526.0,2191.0,41727.0,33.93911,67.709953
2,Afghanistan,01/02/2021,51526.0,2191.0,41727.0,33.93911,67.709953
3,Afghanistan,01/03/2021,51526.0,2191.0,41727.0,33.93911,67.709953
4,Afghanistan,01/04/2021,53011.0,2237.0,42530.0,33.93911,67.709953


In [3]:
country_cols = ["Country/Region", "Latitud_promedio", "Longitud_promedio"]

countries = (
    df[country_cols]
    .dropna(subset=["Latitud_promedio", "Longitud_promedio"])
    .drop_duplicates(subset=["Country/Region"], keep="first")
    .reset_index(drop=True)
)

print(f"Países con coordenadas únicas: {len(countries):,}")
countries.head()


Países con coordenadas únicas: 226


,Country/Region,Latitud_promedio,Longitud_promedio
0,Azerbaijan,40.14310,47.576900
1,Afghanistan,33.93911,67.709953
2,Albania,41.15330,20.168300
3,Algeria,28.03390,1.659600
4,Andorra,42.50630,1.521800


In [4]:
# Función haversine para distancia entre dos puntos geográficos
RADIUS_EARTH_KM = 6371.0

def haversine(lat1, lon1, lat2, lon2):
    """Devuelve la distancia de gran círculo en kilómetros."""
    lat1_rad, lon1_rad = radians(lat1), radians(lon1)
    lat2_rad, lon2_rad = radians(lat2), radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return RADIUS_EARTH_KM * c

# Calculamos todas las distancias pairwise
pairs = []
for (idx_a, row_a), (idx_b, row_b) in combinations(countries.iterrows(), 2):
    dist_km = haversine(
        row_a["Latitud_promedio"], row_a["Longitud_promedio"],
        row_b["Latitud_promedio"], row_b["Longitud_promedio"]
    )
    pairs.append((row_a["Country/Region"], row_b["Country/Region"], dist_km))

pairs[:5], len(pairs)


([('Azerbaijan', 'Afghanistan', 1910.1213118294238),
  ('Azerbaijan', 'Albania', 2305.5325839754923),
  ('Azerbaijan', 'Algeria', 4384.207505643105),
  ('Azerbaijan', 'Andorra', 3807.3395115411035),
  ('Azerbaijan', 'Angola', 6478.034921912277)],
 25425)

In [5]:
distances = np.array([p[2] for p in pairs])
max_distance = distances.max()
min_distance = distances.min()
print(f"Distancia máxima observada: {max_distance:,.2f} km")
print(f"Distancia mínima observada: {min_distance:,.2f} km")

# Normalizamos: closeness = 1 - (distance / max_distance)
# Esto garantiza valores en [0, 1]; distancias muy grandes → valores cercanos a 0.
normalized_pairs = []
for country_a, country_b, dist_km in pairs:
    closeness = 1 - (dist_km / max_distance)
    normalized_pairs.append((country_a, country_b, dist_km, closeness))

normalized_pairs[:5]


Distancia máxima observada: 19,955.87 km
Distancia mínima observada: 0.00 km


[('Azerbaijan', 'Afghanistan', 1910.1213118294238, 0.9042827268316882),
 ('Azerbaijan', 'Albania', 2305.5325839754923, 0.8844684414690553),
 ('Azerbaijan', 'Algeria', 4384.207505643105, 0.7803048503540918),
 ('Azerbaijan', 'Andorra', 3807.3395115411035, 0.8092120360032764),
 ('Azerbaijan', 'Angola', 6478.034921912277, 0.6753819590543825)]

In [6]:
# Construimos un grafo no dirigido y completamente conectado
G = nx.Graph()

# Agregamos nodos con atributos de coordenadas
for _, row in countries.iterrows():
    G.add_node(
        row["Country/Region"],
        lat=row["Latitud_promedio"],
        lon=row["Longitud_promedio"]
    )

# Agregamos aristas con atributos de distancia y cercanía normalizada
for country_a, country_b, dist_km, closeness in normalized_pairs:
    G.add_edge(
        country_a,
        country_b,
        distance_km=dist_km,
        proximity=closeness
    )

print(f"Número de nodos: {G.number_of_nodes()}")
print(f"Número de aristas: {G.number_of_edges()}")
print(f"Grado promedio: {np.mean([deg for _, deg in G.degree()]):.2f}")


Número de nodos: 226
Número de aristas: 25425
Grado promedio: 225.00


In [7]:
# Estadísticas básicas de los pesos de proximidad
proximities = [data["proximity"] for _, _, data in G.edges(data=True)]
print(f"Proximidad mínima: {min(proximities):.4f}")
print(f"Proximidad máxima: {max(proximities):.4f}")
print(f"Proximidad media: {np.mean(proximities):.4f}")

# Muestra de algunas aristas (país, país, distancia km, proximidad)
edge_samples = []
for u, v, data in list(G.edges(data=True))[:10]:
    edge_samples.append({
        "Pais_A": u,
        "Pais_B": v,
        "Distancia_km": data["distance_km"],
        "Proximidad": data["proximity"]
    })

pd.DataFrame(edge_samples)


Proximidad mínima: 0.0000
Proximidad máxima: 1.0000
Proximidad media: 0.6134


,Pais_A,Pais_B,Distancia_km,Proximidad
0,Azerbaijan,Afghanistan,1910.121312,0.904283
1,Azerbaijan,Albania,2305.532584,0.884468
2,Azerbaijan,Algeria,4384.207506,0.780305
3,Azerbaijan,Andorra,3807.339512,0.809212
4,Azerbaijan,Angola,6478.034922,0.675382
5,Azerbaijan,Antigua and Barbuda,10347.120622,0.481500
6,Azerbaijan,Argentina,14244.816591,0.286184
7,Azerbaijan,Armenia,216.060096,0.989173
8,Azerbaijan,Aruba,11325.100201,0.432493
9,Azerbaijan,Australia,12527.652731,0.372232


In [8]:
# Opcional: exportar las aristas a un CSV para análisis externo
edges_df = pd.DataFrame([
    {
        "Pais_A": u,
        "Pais_B": v,
        "Distancia_km": data["distance_km"],
        "Proximidad": data["proximity"]
    }
    for u, v, data in G.edges(data=True)
])

EDGE_OUTPUT_PATH = "data/country_proximity_edges.csv"
edges_df.to_csv(EDGE_OUTPUT_PATH, index=False)
print(f"Aristas exportadas a {EDGE_OUTPUT_PATH}")
edges_df.head()


Aristas exportadas a data/country_proximity_edges.csv


,Pais_A,Pais_B,Distancia_km,Proximidad
0,Azerbaijan,Afghanistan,1910.121312,0.904283
1,Azerbaijan,Albania,2305.532584,0.884468
2,Azerbaijan,Algeria,4384.207506,0.780305
3,Azerbaijan,Andorra,3807.339512,0.809212
4,Azerbaijan,Angola,6478.034922,0.675382


## Resumen

- Dataset base: `data/covid_19_country_daily_with_coords.csv`
- Países con coordenadas: coinciden con el conjunto limpio del notebook de preparación.
- Distancias calculadas mediante fórmula de Haversine.
- Grafo resultante: totalmente conectado, 226 nodos (según tu dataset) y ~25K aristas.
- Peso de cada arista `proximity = 1 - (distancia / distancia_max)` → valores en [0, 1].
- Archivo exportado con todas las aristas: `data/country_proximity_edges.csv`.

Este grafo está listo para análisis de redes (por ejemplo, detectar clústeres geográficos, calcular centralidades, etc.).
